# Detailed Analysis & Debugging Notebook

This notebook provides a step-by-step breakdown of the lattice QCD analysis pipeline. It allows you to inspect the data, visualize the fits, and tune parameters for `r0` and `r0_chi` extraction.

## Instructions
1. Set the **Data Path** and **Parameters** in the "Configuration" section.
2. Run the cells sequentially to initialize data.
3. Use the **Interactive Dashboard** at the end to visualize and debug calculations.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.optimize import curve_fit
from pathlib import Path

# Local modules
import data_organizer as do
from calculator import Calculator, exponential_ansatz, cornell_potential_ansatz

# Setup plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# --- CONFIGURATION ---

# 1. Data Path (Change this to the DIRECTORY containing your .out files)
# Example: "../data/20251221/45" 
FILE_PATH = "../data/20251221/45" # REPLACE THIS WITH YOUR ACTUAL DIRECTORY PATH

# 2. Analysis Parameters
THERMALIZATION_STEPS = 500
SOMMER_TARGET_FORCE = 1.65

# 3. Fitting Parameters
V_R_T_MIN = 1          # Start time for V(R) fits
R0_T_MIN = 2           # t_min parameter passed to r0 calculation (if used)
R0_CHI_T_LARGE = 4     # Large T for Creutz ratio force extraction
CHI_FIT_WINDOW = 2     # Window size for r0_chi interpolation

print(f"Analyzing Directory: {FILE_PATH}")
print(f"Parameters: Therm={THERMALIZATION_STEPS}, Target={SOMMER_TARGET_FORCE}")

## 1. Load Data and Thermalization

We load the raw data using `data_organizer`. We then strip the initial thermalization steps.

In [ ]:
# Load Data
file_data = None
calc = None

try:
    if not Path(FILE_PATH).exists():
        print(f"WARNING: Path {FILE_PATH} not found. Please check the path.")
    else:
        print(f"Loading data from {FILE_PATH}...")
        exp_data = do.ExperimentData(FILE_PATH)
        
        if "W_temp" in exp_data.data and exp_data.data["W_temp"]:
            file_data = exp_data.data["W_temp"][0]
            print(f"Successfully loaded W_temp data: {file_data.name}")
            print(f"Found {len(file_data.observables)} observables.")
            
            # Thermalization Check Plot
            target_obs = next((o for o in file_data.observables if "plaquette" in o.name or "W_temp" in o.name), None)
            step_obs = next((o for o in file_data.observables if o.name in ["# step", "step"]), None)
            
            if target_obs:
                vals = target_obs.values
                # Use step values for X axis if available, otherwise indices
                x_vals = step_obs.values if step_obs else range(len(vals))
                
                plt.figure(figsize=(10,4))
                #only dots not line
                plt.plot(x_vals, vals, label='Raw History', marker='.', linestyle='None', alpha=0.6)
                plt.axvline(THERMALIZATION_STEPS, color='r', linestyle='--', label=f'Cut = {THERMALIZATION_STEPS}')
                
                # Zoom in to 0 - 2*cut
                if THERMALIZATION_STEPS > 0:
                    plt.xlim(0, THERMALIZATION_STEPS * 2)
                    
                plt.title(f"Thermalization Check: {target_obs.name}")
                plt.xlabel("Step" if step_obs else "Index")
                plt.ylabel("Value")
                plt.legend()
                plt.show()
                
                # Apply Cut
                file_data.remove_thermalization(THERMALIZATION_STEPS)
                print(f"Data after thermalization cut: {len(target_obs.values)} samples.")
            else:
                print("Could not find standard observables to plot history.")
        else:
            print(f"ERROR: No 'W_temp' files found in {FILE_PATH}.")
            
except Exception as e:
    print(f"Error loading data: {e}")

## 2. Autocorrelation Analysis

We calculate the integrated autocorrelation time $\tau_{int}$ to determine the appropriate block size for bootstrapping.

In [ ]:
if file_data:
    # Initialize Calculator for Pre-analysis
    calc_pre = Calculator(file_data)
    
    try:
        tau_var = calc_pre.get_variable("tau_int", obs_name="plaquette")
        tau = tau_var.get()
        print(f"Calculated tau_int: {tau:.2f}")
    except Exception as e:
        print(f"Could not calc tau (using default 0.5): {e}")
        tau = 0.5

    BLOCK_SIZE = max(1, int(np.ceil(2 * tau)))
    print(f"Using Block Size: {BLOCK_SIZE}")
    
    # Initialize Main Calculator with Blocking
    calc = Calculator(file_data, step_size=BLOCK_SIZE)
else:
    print("No data loaded.")

## 3. Interactive Analysis Dashboard

Use the tabs below to switch between different analysis steps.
*   **Wilson Loop**: Inspect $W(R,T)$ and effective mass.
*   **Static Potential**: Fit $V(R)$ and determine $r_0$.
*   **Creutz Ratio**: Determine force from ratios and $r_{0,\chi}$.

## 3. Wilson Loops $W(R, T)$ and Potential $V(R)$ Extraction

Here we inspect the raw Wilson loops and the extraction of the potential $V(R)$.
$V(R)$ is extracted by fitting $W(R, T) \approx C e^{-V(R)T}$.

**Adjust `DEBUG_R` below to inspect a specific spatial separation.**

In [ ]:
# Interactive Wilson Loop Analysis
def interact_wilson(R, t_min):
    if not file_data:
        print("No data loaded.")
        return

    # 1. Get W(R, T)
    all_L = np.array(file_data.get("L").values)
    all_T = np.array(file_data.get("T").values)
    unique_ts = sorted(np.unique(all_T[all_L == R]))
    
    ws = []
    ts = []
    w_errs = []
    
    for t in unique_ts:
        try:
            w = calc.get_variable("W_R_T", R=R, T=t)
            ws.append(w.get())
            w_errs.append(w.err())
            ts.append(t)
        except Exception: pass
            
    ws = np.array(ws)
    ts = np.array(ts)
    w_errs = np.array(w_errs)
    
    if len(ws) == 0:
        print(f"No data for R={R}")
        return

    # 2. Get V(R) Fit
    try:
        v_var = calc.get_variable("V_R", R=R, t_min=t_min)
        v_val = v_var.get()
        v_err = v_var.err()
        
        print(f"Fit Result for R={R}, t_min={t_min}:")
        print(f"V(R) = {v_val:.5f} +/- {v_err:.5f}")
        
        # Fit curve
        mask = (ts >= t_min) & (ws > 0)
        ts_fit = ts[mask]
        ws_fit = ws[mask]
        errs_fit = w_errs[mask]
        
        popt = [0, 0]
        if len(ts_fit) > 1:
            try:
                # Re-fit for plotting curve
                p0_V = v_val if not np.isnan(v_val) else 0.5
                p0_C = ws_fit[0] * np.exp(p0_V * ts_fit[0])
                popt, _ = curve_fit(exponential_ansatz, ts_fit, ws_fit, p0=[p0_C, p0_V], sigma=errs_fit, absolute_sigma=True)
            except: pass

        # PLOTS
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Linear Plot
        ax = axes[0]
        ax.errorbar(ts, ws, yerr=w_errs, fmt='o', label='Data')
        t_plot = np.linspace(min(ts), max(ts), 100)
        if popt[1] != 0:
            ax.plot(t_plot, exponential_ansatz(t_plot, *popt), 'r-', label=f'Fit V={popt[1]:.3f}')
        ax.set_title(f"W(R={R}, T) Linear")
        ax.set_xlabel("T")
        ax.legend()
        
        # Log Plot
        ax = axes[1]
        ax.errorbar(ts, ws, yerr=w_errs, fmt='o', label='Data')
        if popt[1] != 0:
            ax.plot(t_plot, exponential_ansatz(t_plot, *popt), 'r-')
        ax.set_yscale('log')
        ax.set_title(f"W(R={R}, T) Log Scale")
        ax.set_xlabel("T")
        
        # Effective Mass
        ax = axes[2]
        meff = []
        meff_t = []
        for i in range(len(ts)-1):
            if ts[i+1] == ts[i] + 1 and ws[i]>0 and ws[i+1]>0:
                val = np.log(ws[i]/ws[i+1])
                meff.append(val)
                meff_t.append(ts[i] + 0.5)
        
        ax.plot(meff_t, meff, 'o-', label='Effective Mass')
        ax.axhline(v_val, color='r', linestyle='--', label=f'V={v_val:.3f}')
        ax.set_title(f"Effective Mass R={R}")
        ax.set_xlabel("T + 0.5")
        ax.set_ylabel("M_eff")
        ax.legend()
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error: {e}")

if file_data:
    unique_Rs = sorted(np.unique(file_data.get("L").values))
    widgets.interact(interact_wilson, 
                     R=widgets.SelectionSlider(options=unique_Rs, description='R', value=unique_Rs[min(2, len(unique_Rs)-1)]),
                     t_min=widgets.IntSlider(min=1, max=10, step=1, value=V_R_T_MIN));
else:
    print("No data.")

## 4. Static Potential $V(R)$ and $r_0$ (Sommer Parameter)

We gather $V(R)$ for all $R$ and fit the Cornell potential:
$$ V(R) = A + \sigma R - \frac{B}{R} $$

The scale $r_0$ is defined where the force $F(r) r^2 = 1.65$.
$$ F(r) = \frac{dV}{dR} = \sigma + \frac{B}{R^2} $$
$$ \implies r_0 = \sqrt{\frac{1.65 - B}{\sigma}} $$

In [ ]:
# Interactive Static Potential Analysis
def interact_potential(t_min, target_force):
    if not file_data: return

    # 1. Collect V(R) using current t_min
    all_L = np.array(file_data.get("L").values)
    unique_Rs = sorted(np.unique(all_L))
    
    rs = []
    vs = []
    v_errs = []
    
    for r in unique_Rs:
        try:
            v_var = calc.get_variable("V_R", R=int(r), t_min=t_min)
            val = v_var.get()
            if not np.isnan(val):
                rs.append(r)
                vs.append(val)
                v_errs.append(v_var.err())
        except: continue
        
    rs = np.array(rs)
    vs = np.array(vs)
    v_errs = np.array(v_errs)
    
    if len(rs) < 3:
        print("Not enough points for V(R)")
        return

    # 2. Fit Cornell & Find r0
    try:
        # We calculate r0 using the Calculator (which handles bootstrapping properly)
        r0_var = calc.get_variable("r0", t_min=t_min, target_force=target_force)
        r0_val = r0_var.get()
        r0_err = r0_var.err()
        
        print(f"r0 (Sommer Parameter) for t_min={t_min}, Target Force={target_force}:")
        print(f"r0 = {r0_val:.4f} +/- {r0_err:.4f}")
        
        # Re-fit for plotting with better initial guess
        sigma_guess = (vs[-1] - vs[0]) / (rs[-1] - rs[0]) if len(rs) > 1 else 0.1
        A_guess = vs[0] - sigma_guess * rs[0]
        p0 = [A_guess, abs(sigma_guess), 0.26]
        
        # Handle small errors to avoid singularity
        fit_sigma = np.array(v_errs)
        fit_sigma[fit_sigma < 1e-12] = 1e-4
        
        try:
            popt, _ = curve_fit(cornell_potential_ansatz, rs, vs, sigma=fit_sigma, absolute_sigma=True, p0=p0, maxfev=5000)
        except Exception as e:
            print(f"Fit Warning: {e}")
            popt = p0

        A, sigma, B = popt
        
        print(f"Cornell Params: A={A:.3f}, sigma={sigma:.3f}, B={B:.3f}")

        # PLOTS
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # V(R) Plot
        ax = axes[0]
        ax.errorbar(rs, vs, yerr=v_errs, fmt='o', label='Data')
        r_plot = np.linspace(min(rs), max(rs), 100)
        ax.plot(r_plot, cornell_potential_ansatz(r_plot, *popt), 'r-', label='Cornell Fit')
        ax.set_title("Static Potential V(R)")
        ax.set_xlabel("R")
        ax.set_ylabel("V(R)")
        ax.legend()
        
        # Force Plot
        ax = axes[1]
        def force_r2(r, sigma, B): return sigma * (r**2) + B
        
        fr2_vals = force_r2(rs, sigma, B)
        ax.plot(rs, fr2_vals, 'o', label='Force from Fit Params')
        ax.plot(r_plot, force_r2(r_plot, sigma, B), 'r-')
        ax.axhline(target_force, color='k', linestyle='--', label=f'Target = {target_force}')
        ax.axvline(r0_val, color='g', linestyle='--', label=f'r0 = {r0_val:.3f}')
        
        ax.set_title("Force Scaling $r^2 F(r)$")
        ax.set_xlabel("R")
        ax.set_ylabel("$r^2 F(r)$")
        ax.legend()
        
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error: {e}")

if file_data:
    widgets.interact(interact_potential,
                     t_min=widgets.IntSlider(min=1, max=5, step=1, value=V_R_T_MIN),
                     target_force=widgets.FloatSlider(min=1.0, max=2.0, step=0.05, value=SOMMER_TARGET_FORCE));

## 5. Creutz Ratios $\chi(R, T)$ and $r_{0,\chi}$

We calculate the Creutz ratios to extract the force directly without fitting a potential.
$$ \chi(R, T) \approx a^2 F(R) $$
We extract the force at a large time $T$.
Then we interpolate $r^2 F(r)$ to find $r_{0,\chi}$.

**Adjust parameters below.**

In [ ]:
# Interactive Creutz Ratio Analysis
def interact_creutz(t_large, target_force, fit_window):
    if not file_data: return

    # 1. Calculate F_chi(R)
    unique_Ls = sorted(np.unique(file_data.get("L").values))
    chi_Rs = [L + 0.5 for L in unique_Ls if L+1 in unique_Ls]
    
    f_chis = []
    f_errs = []
    valid_rs = []
    
    for r in chi_Rs:
        try:
            f_var = calc.get_variable("F_chi", R=r, t_large=t_large)
            val = f_var.get()
            err = f_var.err()
            if not np.isnan(val) and val > 0 and err/val < 0.5:
                f_chis.append(val)
                f_errs.append(err)
                valid_rs.append(r)
        except: continue
        
    valid_rs = np.array(valid_rs)
    f_chis = np.array(f_chis)
    f_errs = np.array(f_errs)
    
    if len(valid_rs) == 0:
        print("No valid Creutz ratios found.")
        return

    # 2. Calculate r0_chi
    try:
        r0c_var = calc.get_variable("r0_chi", t_large=t_large, target_force=target_force, fit_window=fit_window)
        r0c_val = r0c_var.get()
        r0c_err = r0c_var.err()
        
        print(f"r0_chi for T={t_large}, Target={target_force}:")
        print(f"r0_chi = {r0c_val:.4f} +/- {r0c_err:.4f}")
        
        # PLOTS
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # Force F(R)
        ax = axes[0]
        ax.errorbar(valid_rs, f_chis, yerr=f_errs, fmt='o', label='Creutz F(R)')
        ax.set_title(f"Creutz Force F(R) at T={t_large}")
        ax.set_xlabel("R")
        ax.set_ylabel("F(R)")
        ax.set_yscale('log')
        ax.legend()
        
        # r^2 F(r) Interpolation
        ax = axes[1]
        r2f = (valid_rs**2) * f_chis
        r2f_err = (valid_rs**2) * f_errs
        
        ax.errorbar(valid_rs, r2f, yerr=r2f_err, fmt='o', label='$r^2 \chi(r)$')
        ax.axhline(target_force, color='k', linestyle='--', label=f'Target = {target_force}')
        
        if not np.isnan(r0c_val):
             ax.axvline(r0c_val, color='g', linestyle='--', label=f'r0_chi = {r0c_val:.3f}')
        
        ax.set_title("Creutz Scale Determination")
        ax.set_xlabel("R")
        ax.set_ylabel("$r^2 F(r)$")
        ax.legend()
        
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error: {e}")

if file_data:
    widgets.interact(interact_creutz,
                     t_large=widgets.IntSlider(min=2, max=10, step=1, value=R0_CHI_T_LARGE),
                     target_force=widgets.FloatSlider(min=1.0, max=2.0, step=0.05, value=SOMMER_TARGET_FORCE),
                     fit_window=widgets.IntSlider(min=1, max=4, step=1, value=CHI_FIT_WINDOW));

## Creutz Physical Ratio P(R)

Here we calculate the physical ratio $P(R)$ defined by Creutz:
$$ P(R) = 1 - \frac{W(R,R) W(R/2,R/2)}{W(R,R/2)^2} $$

This quantity is designed to be free of perimeter and corner divergences.

In [ ]:
def analyze_creutz_P():
    if not 'calc' in globals():
        print("Calculator not initialized. Please run previous cells.")
        return

    # Try even R values
    rs = []
    p_vals = []
    p_errs = []

    print("Calculating Creutz P(R):")
    for R in range(2, 7, 2): # 2, 4, 6,
        try:
            # We need W(R,R), W(R/2,R/2), W(R, R/2)
            # The calculator handles the retrieval.
            p_var = calc.get_variable("creutz_P", R=R)
            val = p_var.get()
            err = p_var.err()
            
            if val is not None and not np.isnan(val):
                print(f"R={R}: P = {val:.6f} +/- {err:.6f}")
                rs.append(R)
                p_vals.append(val)
                p_errs.append(err if err is not None else 0.0)
            else:
                 print(f"R={R}: Result is NaN or None")

        except Exception as e:
            # Likely missing Wilson loops for large R
            print(f"R={R}: Could not calculate ({e})")
    
    if not rs:
        print("No P(R) values could be calculated.")
        return

    # Plot
    plt.figure(figsize=(8, 6))
    plt.errorbar(rs, p_vals, yerr=p_errs, fmt='o-', capsize=5, label='P(R)')
    plt.title("Creutz Physical Ratio P(R)")
    plt.xlabel("R (lattice units)")
    plt.ylabel("P(R)")
    plt.grid(True)
    plt.legend()
    plt.show()

analyze_creutz_P()

In [ ]:
def analyze_lattice_spacing():
    if not 'calc' in globals():
        print("Calculator not initialized. Please run previous cells.")
        return

    rs = []
    a_vals = []
    a_errs = []
    
    sigma_val = 440.0 # MeV

    print(f"Calculating Lattice Spacing a (assuming sqrt(sigma)={sigma_val} MeV):")
    for R in range(2, 12, 2):
        try:
            # Calculate a using the new calculator method
            a_var = calc.get_variable("a_creutz", R=R, sigma_sqrt_MeV=sigma_val)
            val = a_var.get()
            err = a_var.err()
            
            if val is not None and not np.isnan(val):
                print(f"R={R}: a = {val:.4f} +/- {err:.4f} fm")
                rs.append(R)
                a_vals.append(val)
                a_errs.append(err if err is not None else 0.0)
            else:
                print(f"R={R}: Result is NaN (likely P(R) out of bounds)")
        except Exception as e:
            # Usually happens if Wilson loops are missing for large R
            pass 
            
    if not rs:
        print("No lattice spacing values calculated.")
        return

    # Plot
    plt.figure(figsize=(8, 6))
    plt.errorbar(rs, a_vals, yerr=a_errs, fmt='o-', capsize=5, label=f'a from P(R)')
    plt.title(f"Lattice Spacing a (assuming $\sqrt{{\sigma}}={sigma_val}$ MeV)")
    plt.xlabel("R (lattice units)")
    plt.ylabel("a [fm]")
    plt.grid(True)
    plt.legend()
    plt.show()

analyze_lattice_spacing()